In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install llama-index
!pip install llama-index-llms-groq
!pip install llama-index-embeddings-huggingface
!pip install llama-parse
!pip install nest_asyncio
!pip install google-colab
!pip install opencv-python
!pip install fastapi uvicorn
!pip install python-multipart
!pip install spacy
!pip install transformers
!python -m spacy download en_core_web_sm

In [ ]:
import nest_asyncio
import os
import pytesseract
from PIL import Image
import cv2
import re
import json
import requests
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.llms import ChatMessage
from llama_index.core import Settings
from transformers import pipeline
import spacy
import time
import threading
from google.colab import files

# Apply asyncio patch
nest_asyncio.apply()

# Environment variables for API keys (replace with real keys)
os.environ["GROQ_API_KEY"] = "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"
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-U6JoUqhtKdQRwx20kGS8sy61gdozYI70EN8P5xWNq3nCNvFr"
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# Initialize LLM and embeddings
llm = Groq(model="llama3-8b-8192")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = llm
Settings.embed_model = embed_model

# Load the spaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load Hugging Face zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Labels for question classification
labels = ["harmful", "health benefits", "general ingredient question"]

# User profiles (stored in a JSON file)
user_profiles_file = "user_profiles.json"

# Load user profiles from JSON
def load_user_profiles():
    if os.path.exists(user_profiles_file):
        with open(user_profiles_file, "r") as file:
            return json.load(file)
    return {}

# Save user profiles to JSON
def save_user_profiles(profiles):
    with open(user_profiles_file, "w") as file:
        json.dump(profiles, file)

# Create or update user profile
def create_or_update_profile(user_id, dietary_restrictions=None, preferences=None):
    profiles = load_user_profiles()
    profiles[user_id] = {
        "dietary_restrictions": dietary_restrictions or profiles.get(user_id, {}).get("dietary_restrictions", []),
        "preferences": preferences or profiles.get(user_id, {}).get("preferences", []),
    }
    save_user_profiles(profiles)
    print(f"Profile updated for user {user_id}: {profiles[user_id]}")

# Extract text from image using Tesseract
def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    extracted_text = pytesseract.image_to_string(gray)
    return extracted_text

# Improved Ingredient Extraction using regex and NLP techniques
def extract_ingredients_from_text(extracted_text):
    cleaned_text = re.sub(r"[^a-zA-Z, ]+", "", extracted_text).lower()
    ingredients = [ingredient.strip() for ingredient in cleaned_text.split(",") if ingredient.strip()]
    return ingredients

# Recommend products based on ingredients and user preferences
def recommend_products_based_on_ingredients(ingredients, user_profile):
    # Placeholder: Implement your API interaction here
    products = []  # Replace with actual API call to fetch products based on ingredients

    recommendations = []
    for product in products:
        if any(allergen in product["ingredients"] for allergen in user_profile.get("dietary_restrictions", [])):
            continue
        if all(pref in product["tags"] for pref in user_profile.get("preferences", [])):
            recommendations.append(product)

    return recommendations

# Function to classify user queries using spaCy and Hugging Face
def classify_user_query(user_question):
    # Use spaCy to process the text
    doc = nlp(user_question)

    # Extract the entities (like ingredients or products)
    entities = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PRODUCT"]]

    # Classify the question using Hugging Face zero-shot classifier
    result = classifier(user_question, candidate_labels=labels)

    # Determine the highest-scoring label
    classification = result["labels"][0]
    score = result["scores"][0]

    print(f"Query classification: {classification} (Confidence: {score:.2f})")

    # Route the query based on the classification
    if classification == "harmful":
        return handle_harmful_question(entities)
    elif classification == "health benefits":
        return handle_health_benefits_question(entities)
    else:
        return handle_general_question(entities)

# Define handlers for different types of questions
def handle_harmful_question(entities):
    if entities:
        return f"Checking whether {entities[0]} is harmful..."
    else:
        return "Please specify an ingredient you're asking about."

def handle_health_benefits_question(entities):
    if entities:
        return f"Here are some health benefits of {entities[0]}..."
    else:
        return "Please specify the ingredient you want to know the health benefits of."

def handle_general_question(entities):
    if entities:
        return f"Here's some general information about {entities[0]}..."
    else:
        return "Please specify the ingredient you're asking about."

# Timeout decorator for user inactivity
def timeout_input(prompt, timeout=600):
    result = [None]

    def input_with_timeout():
        result[0] = input(prompt)

    thread = threading.Thread(target=input_with_timeout)
    thread.daemon = True
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        return None
    return result[0]

# Function for continuous chat with a basic chatbot
def continuous_chat_with_basic_chatbot(extracted_text):
    while True:
        user_question = timeout_input("\nYou: ")  # Get user input with a 10-minute timeout
        if user_question is None:
            print("\nChat ended due to inactivity.")
            break
        if user_question.lower() in ["exit", "quit", "main"]:
            print("Returning to main menu.")
            break

        # Classify the user question and route it to the appropriate handler
        response = classify_user_query(user_question)
        print("\nIngredients Assistant: ", response)

# Function for continuous chat with LLaMA (natural conversation)
def continuous_chat_with_llama(extracted_text):
    messages = [
        ChatMessage(role="system", content="You are an AI assistant who provides detailed information and has natural conversations."),
        ChatMessage(role="user", content=f"Here is some extracted text: {extracted_text}. Can you explain what this is about?")
    ]

    # Initial chat response from LLaMA
    chat_response = llm.chat(messages)
    print("\nInitial Shopping Assistant Response:\n", chat_response)

    while True:
        user_question = timeout_input("\nYou: ")  # Get user input with a 10-minute timeout
        if user_question is None:
            print("\nChat ended due to inactivity.")
            break
        if user_question.lower() in ["exit", "quit", "main"]:
            print("Returning to main menu.")
            break

        # Continue the conversation naturally with LLaMA
        messages.append(ChatMessage(role="user", content=user_question))
        chat_response = llm.chat(messages)
        print("\nShopping: ", chat_response)

# Main menu for chatbot selection
def main_menu():
    while True:
        uploaded = files.upload()
        user_id = input("Enter your user ID: ")

        for fn in uploaded.keys():
            extracted_text = extract_text_from_image(fn)
            print("Extracted Text:", extracted_text)

            update_profile = input("Would you like to update your profile? (yes/no): ").lower()
            if update_profile == "yes":
                dietary_restrictions = input("Enter dietary restrictions (comma-separated): ").split(',')
                preferences = input("Enter preferences (comma-separated): ").split(',')
                create_or_update_profile(user_id, dietary_restrictions=dietary_restrictions, preferences=preferences)

            # Provide user the option to choose between continuous chat with LLaMA or a basic chatbot
            while True:
                chat_option = input("Choose a chatbot option: (1) Ingredients Assistant (2) Shopping Assistant\nEnter your choice (1, 2, or 'exit' to quit): ").strip()

                if chat_option == "1":
                    # Begin continuous chat with basic chatbot
                    continuous_chat_with_basic_chatbot(extracted_text)
                elif chat_option == "2":
                    # Begin continuous chat with LLaMA
                    continuous_chat_with_llama(extracted_text)
                elif chat_option.lower() == "exit":
                    print("Exiting program.")
                    return
                else:
                    print("Invalid choice! Please try again. To Go Back to Main Menu Enter 'main'")

# Run the main menu function
main_menu()
